In [1]:
import pandas as pd 
import requests
import warnings
warnings.filterwarnings('ignore')

# Utility functions

In [2]:
def preprocess_data_per_year(data,df_department,year):
    # Get data relative to the year entered
    df_elec=data.loc[data["Année"]==year]
    # Remove dupplicate values for columns ...
    df_elec.drop_duplicates(subset=['Code INSEE de la commune'],inplace=True)
    # Drop missing values 
    df_elec.dropna(subset=['Code INSEE de la commune', 'Consommation annuelle moyenne de la commune (MWh)', 'Nom de la commune'],inplace=True)
    df_elec.rename(columns={'Code INSEE de la commune': 'code_commune_INSEE'}, inplace=True)
    df_elec['code_commune_INSEE']=df_elec['code_commune_INSEE'].astype(str)
    INSEE_codes=set(df_elec['code_commune_INSEE'])
    df_department=df_department.loc[df_department['code_commune_INSEE'].isin(INSEE_codes)]
    df_department['code_commune_INSEE']=df_department['code_commune_INSEE'].astype(int)
    df_elec['code_commune_INSEE']=df_elec['code_commune_INSEE'].astype(int)
    # Merging the two dataframes
    df_elec_department=pd.merge(df_department, df_elec, how='inner', on='code_commune_INSEE')
    df_elec_department.drop_duplicates(subset=['Consommation annuelle moyenne de la commune (MWh)', 'Nom de la commune', 'nom_departement'],inplace=True)
    df_elec_department.dropna(subset=[ 'Nom de la commune'],inplace=True)
    df_elec_department=df_elec_department[['Consommation annuelle moyenne de la commune (MWh)', 'Nom de la commune', 'nom_departement']]
    df_elec_department=df_elec_department.groupby(['nom_departement']).sum()
    return df_elec_department

# Importing data

In [3]:
df_department=pd.read_csv("communes-departement-region.csv")
data= pd.read_csv('datamap.csv',  sep=';')

## Creating data per department with function 

In [4]:
cons={}
for year in range(2018,2022): 
    data_year=preprocess_data_per_year(data, df_department,year)
    cons[str(year)] =data_year["Consommation annuelle moyenne de la commune (MWh)"].values
df_cons_per_depart=pd.DataFrame(cons)
df_cons_per_depart["departement"]=data_year.index.values
df_cons_per_depart['avg_consumption'] = df_cons_per_depart.mean(axis=1)

## Print the data frame clean

In [5]:
df_cons_per_depart

,2018,2019,2020,2021,departement,avg_consumption
0,806.270,843.413,880.157,911.311,Ain,860.28775
1,354.479,379.110,374.006,383.455,Aisne,372.76250
2,148.999,159.072,168.750,181.198,Allier,164.50475
3,512.908,498.858,521.022,513.162,Alpes-Maritimes,511.48750
4,157.925,165.898,158.664,163.284,Alpes-de-Haute-Provence,161.44275
...,...,...,...,...,...,...
88,265.122,338.884,428.466,361.753,Vendée,348.55625
89,88.716,94.814,99.623,93.163,Vienne,94.07900
90,305.399,316.104,314.855,338.797,Vosges,318.78875
91,180.799,175.844,164.595,169.548,Yonne,172.69650


## Tranforming data to csv file 

In [6]:
df_cons_per_depart[['departement','avg_consumption']].to_csv("data_final.csv", index=False)